# MOSE playground

## Basic Arguments Settting

In [2]:
import argparse
import random
import numpy as np
import torch
import os
import warnings
from multi_runs import multiple_run

import datetime
import numpy as np
import torch
import torch.nn.functional as F
from torch.optim import Adam, SGD
from experiment.dataset import get_data
from models.buffer import Buffer
from train_scd import TrainLearner_SCD
from models.Resnet18_SD import resnet18_sd
from utils.util import compute_performance

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
warnings.filterwarnings('ignore')

args = argparse.Namespace()
setattr(args, 'seed', 0)
setattr(args, 'lr', 1e-3)
setattr(args, 'wd', 1e-4)
setattr(args, 'dataset', 'cifar100')
setattr(args, 'buffer_size', 5000)
setattr(args, 'buffer_batch_size', 64)
setattr(args, 'run_nums', 1)
setattr(args, 'batch_size', 10)
setattr(args, 'ins_t', 0.07)
setattr(args, 'gpu_id', 5)
setattr(args, 'n_workers', 8)

torch.cuda.set_device(args.gpu_id)
args.cuda = torch.cuda.is_available()
print('=' * 100)
print('Arguments =')
for arg in vars(args):
    print('\t' + arg + ':', getattr(args, arg))
np.random.seed(args.seed)
random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    print('[CUDA is unavailable]')

Arguments =
	seed: 0
	lr: 0.001
	wd: 0.0001
	dataset: cifar100
	buffer_size: 5000
	buffer_batch_size: 64
	run_nums: 1
	batch_size: 10
	ins_t: 0.07
	gpu_id: 5
	n_workers: 8
	cuda: True


## Multiple Run

In [ ]:
test_all_acc = torch.zeros(args.run_nums)
accuracy_list = []

for run in range(args.run_nums):
    tmp_acc = []
    print('=' * 100)
    print(f"-----------------------------run {run} start--------------------------")
    print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    print('=' * 100)

    # initialize
    data, class_num, class_per_task, task_loader, input_size = get_data(args.dataset, args.batch_size, args.n_workers)
    args.n_classes = class_num
    buffer = Buffer(args, input_size).cuda()

    model = resnet18_sd(class_num).cuda()
    optimizer = Adam(model.parameters(), lr=args.lr,  weight_decay=1e-4)
    agent = TrainLearner_SCD(model, buffer, optimizer, class_num, class_per_task, input_size, args)

    # training
    for i in range(len(task_loader)):
        print(f"-----------------------------run {run} task id:{i} start training-----------------------------")

        agent.train(i, task_loader[i]['train'])
        acc_list = agent.test(i, task_loader)
        tmp_acc.append(acc_list)

    test_accuracy = acc_list.mean()
    test_all_acc[run] = test_accuracy
    accuracy_list.append(np.array(tmp_acc))

    print('=' * 100)
    print("{}th run's Test result: Accuracy: {:.2f}%".format(run, test_accuracy))
    print('=' * 100)

    agent.save_checkpoint('./outputs/final.pt')

accuracy_array = np.array(accuracy_list)
avg_end_acc, avg_end_fgt, avg_acc, avg_bwtp, avg_fwt = compute_performance(accuracy_array)
print('=' * 100)
print(f"total {args.run_nums}runs test acc results: {test_all_acc}")
print('----------- Avg_End_Acc {} Avg_End_Fgt {} Avg_Acc {} Avg_Bwtp {} Avg_Fwt {}-----------'
        .format(avg_end_acc, avg_end_fgt, avg_acc, avg_bwtp, avg_fwt))
print('=' * 100)

## Test Ensemble Outputs

In [3]:
# initialize
data, class_num, class_per_task, task_loader, input_size = get_data(args.dataset, args.batch_size, args.n_workers)
args.n_classes = class_num
buffer = Buffer(args, input_size).cuda()
model = resnet18_sd(class_num).cuda()
optimizer = Adam(model.parameters(), lr=args.lr,  weight_decay=1e-4)
agent = TrainLearner_SCD(model, buffer, optimizer, class_num, class_per_task, input_size, args)

Task order = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Data and loader is prepared
buffer has 5000 slots
bx torch.Size([5000, 3, 32, 32])


In [4]:
# loading
agent.load_checkpoint('./outputs/final.pt')
agent.buffer.current_index = args.buffer_size - 1

Load checkpoint from: ./outputs/final.pt


In [5]:
def test(agent, i, task_loader, feat_ids=[3]):
    agent.model.eval()
    acc_lists = []
    for feat_id in feat_ids:
        print(f"{'*'*100}\nTest with the output of layer: {feat_id+1}\n")
        agent.class_means = {}
        agent.seen_classes = list(set(agent.buffer.y_int.tolist()))
        class_inputs = {cls: [] for cls in agent.seen_classes}
        for x, y in zip(agent.buffer.x, agent.buffer.y_int):
            class_inputs[y.item()].append(x)

        for cls, inputs in class_inputs.items():
            features = []
            for ex in inputs:
                # feature = agent.model.final_feature(ex.unsqueeze(0)).detach().clone()
                feature = agent.model.features(ex.unsqueeze(0))[feat_id].detach().clone()
                feature = F.normalize(feature, dim=1)
                features.append(feature.squeeze())

            if len(features) == 0:
                mu_y = torch.normal(
                    # 0, 1, size=tuple(agent.model.final_feature(x.unsqueeze(0)).detach().size())
                    0, 1, size=tuple(agent.model.features(x.unsqueeze(0))[feat_id].detach().size())
                )
                mu_y = mu_y.to(x.device)
            else:
                features = torch.stack(features)
                mu_y = features.mean(0)
            mu_y = F.normalize(mu_y.reshape(1, -1), dim=1)
            agent.class_means[cls] = mu_y.squeeze()

        with torch.no_grad():
            acc_list = np.zeros(len(task_loader))
            for j in range(i + 1):
                acc = agent.test_model(task_loader[j]['test'], j, feat_id=feat_id)
                acc_list[j] = acc.item()

            print(f"tasks acc:{acc_list}")
            print(f"tasks avg acc:{acc_list[:i+1].mean()}")
        acc_lists.append(acc_list)

    return acc_lists

In [6]:
# get results of all layers
acc_lists = test(agent, 9, task_loader, feat_ids=[0,1,2,3])

****************************************************************************************************
Test with the output of layer: 1

Test task 0: Accuracy: 509/1000 (50.90%)
Test task 1: Accuracy: 446/1000 (44.60%)
Test task 2: Accuracy: 504/1000 (50.40%)
Test task 3: Accuracy: 417/1000 (41.70%)
Test task 4: Accuracy: 481/1000 (48.10%)
Test task 5: Accuracy: 525/1000 (52.50%)
Test task 6: Accuracy: 473/1000 (47.30%)
Test task 7: Accuracy: 484/1000 (48.40%)
Test task 8: Accuracy: 503/1000 (50.30%)
Test task 9: Accuracy: 495/1000 (49.50%)
tasks acc:[50.90000153 44.59999847 50.40000153 41.70000076 48.09999847 52.5
 47.29999924 48.40000153 50.29999924 49.5       ]
tasks avg acc:48.37000007629395
****************************************************************************************************
Test with the output of layer: 2

Test task 0: Accuracy: 487/1000 (48.70%)
Test task 1: Accuracy: 477/1000 (47.70%)
Test task 2: Accuracy: 552/1000 (55.20%)
Test task 3: Accuracy: 434/1000 (43.40%

In [7]:
with open('./outputs/acc_lists.txt', 'w') as f:
    for acc_list in acc_lists:
        for acc in acc_list:
            f.write(f"{acc:.2f}\t")
        f.write('\n')

In [8]:
acc_lists = np.array(acc_lists)
print(acc_lists.shape)
opt_acc_list = acc_lists.max(axis=0)
print(opt_acc_list)
print(opt_acc_list.mean())

(4, 10)
[50.90000153 47.70000076 55.20000076 43.40000153 51.79999924 54.29999924
 49.70000076 52.79999924 57.         62.70000076]
52.55000038146973


In [ ]:
agent.model.eval()
class_means_ls = [{} for _ in range(4)]
seen_classes = list(set(agent.buffer.y_int.tolist()))
class_inputs = {cls: [] for cls in seen_classes}

for x, y in zip(agent.buffer.x, agent.buffer.y_int):
    class_inputs[y.item()].append(x)

for cls, inputs in class_inputs.items():
    features = [[] for _ in range(4)]
    for ex in inputs:
        features_ls = agent.model.features(ex.unsqueeze(0))
        for feat_id in range(4):
            feature = features_ls[feat_id].detach().clone()
            feature = F.normalize(feature, dim=1)
            features[feat_id].append(feature.squeeze())

    for feat_id in range(4):
        if len(features[feat_id]) == 0:
            mu_y = torch.normal(
                0, 1, size=tuple(agent.model.features(x.unsqueeze(0))[feat_id].detach().size())
            )
            mu_y = mu_y.to(x.device)
        else:
            features[feat_id] = torch.stack(features[feat_id])
            mu_y = features[feat_id].mean(0)
        
        mu_y = F.normalize(mu_y.reshape(1, -1), dim=1)
        class_means_ls[feat_id][cls] = mu_y.squeeze()


In [ ]:
torch.save(class_means_ls, './outputs/class_means_ls.pt')

In [10]:
class_means_ls = torch.load('./outputs/class_means_ls.pt')

In [19]:
# test ncm classifier
seen_classes = list(set(agent.buffer.y_int.tolist()))
agent.model.eval()
for feat_id in range(4):
    print(f"\nTest with feature layer {feat_id}")
    class_means = class_means_ls[feat_id]
    with torch.no_grad():
        acc_list = np.zeros(len(task_loader))
        for j in range(10):
            correct = torch.full([], 0).cuda()
            num = torch.full([], 0).cuda()
            for batch_idx, (data, target) in enumerate(task_loader[j]['test']):
                data, target = data.cuda(), target.cuda()
                features = agent.model.features(data)[feat_id]
                features = F.normalize(features, dim=1)
                features = features.unsqueeze(2)
                means = torch.stack([class_means[cls] for cls in seen_classes])
                means = torch.stack([means] * data.size(0))
                means = means.transpose(1, 2)
                features = features.expand_as(means)
                dists = (features - means).pow(2).sum(1).squeeze()
                pred = dists.min(1)[1]
                pred = torch.Tensor(seen_classes)[pred].to(data.device)

                num += data.size()[0]
                correct += pred.eq(target.data.view_as(pred)).sum()

            test_accuracy = (100. * correct / num)
            acc_list[j] = test_accuracy
            print('Test task {}: Accuracy: {}/{} ({:.2f}%)'.format(j, correct, num, test_accuracy))
    print(f"Average End Accuracy: {acc_list.mean():.2f}%")


Test with feature layer 0


Test task 0: Accuracy: 509/1000 (50.90%)
Test task 1: Accuracy: 446/1000 (44.60%)
Test task 2: Accuracy: 504/1000 (50.40%)
Test task 3: Accuracy: 417/1000 (41.70%)
Test task 4: Accuracy: 481/1000 (48.10%)
Test task 5: Accuracy: 525/1000 (52.50%)
Test task 6: Accuracy: 473/1000 (47.30%)
Test task 7: Accuracy: 484/1000 (48.40%)
Test task 8: Accuracy: 503/1000 (50.30%)
Test task 9: Accuracy: 495/1000 (49.50%)
Average End Accuracy: 48.37%

Test with feature layer 1
Test task 0: Accuracy: 487/1000 (48.70%)
Test task 1: Accuracy: 477/1000 (47.70%)
Test task 2: Accuracy: 552/1000 (55.20%)
Test task 3: Accuracy: 434/1000 (43.40%)
Test task 4: Accuracy: 518/1000 (51.80%)
Test task 5: Accuracy: 532/1000 (53.20%)
Test task 6: Accuracy: 486/1000 (48.60%)
Test task 7: Accuracy: 485/1000 (48.50%)
Test task 8: Accuracy: 521/1000 (52.10%)
Test task 9: Accuracy: 531/1000 (53.10%)
Average End Accuracy: 50.23%

Test with feature layer 2
Test task 0: Accuracy: 495/1000 (49.50%)
Test task 1: Accuracy: 454/

In [25]:
# test ncm classifier
seen_classes = list(set(agent.buffer.y_int.tolist()))
agent.model.eval()
with torch.no_grad():
    acc_list = np.zeros(len(task_loader))
    for j in range(10):
        correct = torch.full([], 0).cuda()
        num = torch.full([], 0).cuda()
        for batch_idx, (data, target) in enumerate(task_loader[j]['test']):
            data, target = data.cuda(), target.cuda()
            features_ls = agent.model.features(data)
            dists_ls = []
            for feat_id in range(4):
                class_means = class_means_ls[feat_id]
                features = features_ls[feat_id]
                features = F.normalize(features, dim=1)
                features = features.unsqueeze(2)
                means = torch.stack([class_means[cls] for cls in seen_classes])
                means = torch.stack([means] * data.size(0))
                means = means.transpose(1, 2)
                features = features.expand_as(means)
                dists = (features - means).pow(2).sum(1).squeeze()
                dists_ls.append(dists)

            dists_ls = torch.cat([dists.unsqueeze(1) for dists in dists_ls], dim=1)
            dists = dists_ls.mean(dim=1).squeeze(1)
            pred = dists.min(1)[1]
            pred = torch.Tensor(seen_classes)[pred].to(data.device)
            num += data.size()[0]
            correct += pred.eq(target.data.view_as(pred)).sum()

        test_accuracy = (100. * correct / num)
        acc_list[j] = test_accuracy
        print('Test task {}: Accuracy: {}/{} ({:.2f}%)'.format(j, correct, num, test_accuracy))
print(f"Average End Accuracy: {acc_list.mean():.2f}%")

Test task 0: Accuracy: 510/1000 (51.00%)
Test task 1: Accuracy: 475/1000 (47.50%)
Test task 2: Accuracy: 556/1000 (55.60%)
Test task 3: Accuracy: 441/1000 (44.10%)
Test task 4: Accuracy: 517/1000 (51.70%)
Test task 5: Accuracy: 561/1000 (56.10%)
Test task 6: Accuracy: 520/1000 (52.00%)
Test task 7: Accuracy: 538/1000 (53.80%)
Test task 8: Accuracy: 601/1000 (60.10%)
Test task 9: Accuracy: 623/1000 (62.30%)
Average End Accuracy: 53.42%


In [ ]:
# test classifier output
agent.model.eval()
# for feat_id in range(4):
with torch.no_grad():
    acc_list = np.zeros(len(task_loader))
    for j in range(9 + 1):
        correct = torch.full([], 0).cuda()
        num = torch.full([], 0).cuda()
        for batch_idx, (data, target) in enumerate(task_loader[j]['test']):
            data, target = data.cuda(), target.cuda()
            # pred = agent.model(data)[feat_id]
            pred = agent.model(data)
            pred = torch.stack(pred, dim=1)
            pred = pred.mean(dim=1).squeeze()
            Pred = pred.data.max(1, keepdim=True)[1]
            num += data.size()[0]
            correct += Pred.eq(target.data.view_as(Pred)).sum()

        test_accuracy = (100. * correct / num)
        print('Test task {}: Accuracy: {}/{} ({:.2f}%)'.format(j, correct, num, test_accuracy))

        acc_list[j] = test_accuracy.item()

    print(f"tasks acc:{acc_list}")
    print(f"tasks avg acc:{acc_list[:9+1].mean()}")

## GSA ce loss modification

In [1]:
import torch
import torch.nn.functional as F

In [3]:
x = torch.randn(128, 100)
y = torch.randn(128, 100)

loss_func = torch.nn.SmoothL1Loss(reduction='mean', beta=1)
print(loss_func(x, y))

tensor(0.7230)


In [4]:
print(torch.dist(x, y, p=2))

tensor(160.2610)


In [5]:
mse = torch.nn.MSELoss(reduction='mean')
print(mse(x,y))

tensor(2.0065)
